Welcome to the laboratory computers for the course "Neural signals and signal processing". The aim of the laboratories is to provide insights on how to analyze other kinds of imaging data: here in particular we will look at functional Near Infrared Spectroscopy (fNRIS) and diffusion-weighted MRI to generate tractography and a structural connectome.

<div class="alert alert-warning">
    <p><b>Please remember to run this notebook always with the command: "fsleyes --notebookFile lab_3_fNIRs_tracto.ipynb"! </b></p>
</div>

<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Neural Signals and Signal Processing (NX-421)</h2>
<hr style="clear:both"></hr>
<h1><font color='black'>Part 1: Get acquainted with fNIRS data </font></h1>


# 1. Vizualisation of neurophysiological signals</font></h1>

fMRI has become the gold standard for in-vivo visualization of the human brain. Nevertheless, this technique is restricted to a laboratory set-up in which the participant is restricted to a fixed position. In contrast, fNIRS experimental set-up stands out for more ecological validity, thanks to a portable version of the device that allows to perform experiments in realistic environments. Still in comparison with fMRI, fNIRS has a relatively higher temporal resolution of 0,01s that allows to disentangle between fluctuations of oxygenated hemoglobin (HbO) and deoxygenated hemoglobin (HbR). But unlike fMRI magnets, fNIRS optical sensors are limited by a low spatial resolution of 3mm and a penetration depth of 1,5mm which does not allow to probe deep areas of the brain.

Given these measurement settings, let's visualize and inspect data generated with fNIRS! 

### Dataset

We will work on a set of hemodynamic data measured on one subject during a finger tapping paradigm with three conditions: 1) Tapping the left thumb to fingers, 2) Tapping the right thumb to fingers and 3) A control when nothing happens. Each tapping lasts 5 seconds and there are 30 trials in each condition.The measurement was performed using fNIRS sensors, which were located over motor areas of the cortex. 

Data were provided by Luke, R., & McAlpine, D. (2021). fNIRS Finger Tapping Data in BIDS Format (Version v0.0.1) (https://doi.org/10.5281/zenodo.5529797),

### Download and load data

Load the dataset by running the next cell.

In [ ]:
import os.path as op
import mne
import mne_nirs
import matplotlib.pyplot as plt

#Download measurement
fnirs_data_folder = mne.datasets.fnirs_motor.data_path()
#Get the path for the measurement folder 
fnirs_data_folder=op.join(fnirs_data_folder, 'Participant-1')
#Load the measurement 
raw_intensity = mne.io.read_raw_nirx(fnirs_data_folder, verbose=True, preload=True)

Display and read information on the measurement setting such as the number of channels, the file duration or the sampling frequency by runing the next cell. 

In [ ]:
#Display information on measurement setting
raw_intensity

##### Multiple Choice Question:
Among the choices below, what is the shape of the measured dataset (number of channels, timepoints) ?
   * 1. (17, 98888),
   * 2. (56, 23239),
   * 3. (56, 78889), or
   * 4. (17, 23239).

Verify your answer by applying the function shape() on your dataset. Beforehand, you need to extract the dataset from the measuerment file using the function get_data(). Look at the mne documentation by clicking on the following link: https://mne.tools/stable/generated/mne.io.Raw.html.

In [ ]:
#YOUR CODE HERE

As another measurement setting, the location of brain sensors is very important to understand the spatial resolution of data. Especially, fNIRS sensors, by only covering predefined regions of the cortex, show a low spatial resolution. 

Run the next cell to view the locations of sensors over the brain surface. 

In [ ]:
#View location of sensors over brain surface
%matplotlib inline
raw_intensity.plot_sensors()

##### Multiple Choice Question
What are the lobes covered by this measurement ?
* 1. Temporal lobe
* 2. Occipital lobe
* 3. Prefrontal lobe 
* 4. Parietal lobe

Let's now have a look at the experimental design used for this measurement.

The experimental designs most commonly employed by auditory fNIRS researchers are block- and event-related designs. Event-related design refers to multiple stimuli that are assumed to occur instantaneously and have randomized time between events. In an event-related finger tapping experiment, the participant would tap a button with the finger approximately every 20 seconds. In a block design, the finger tapping would be performed continuously during a certain time interval and would be followed by a short block of rest.

In turn, the choice of the experimental design depends on a range of factors, including the statistical power of the protocol, the duration of the experiment, and whether the design provides the flexibility to study the effect of interest. While the block design might lead to higher detection power, it can also induce learning and boredom effects which may bias the results. On the other hand, event-related designs reduce the effects of learning, boredom and other events unrelated to the task while exhibiting loss in detection power. 

Run the next cell to display the sequence of events implemented in this measurement.

In [ ]:
#Load the event annotations
events, event_dict=mne.events_from_annotations(raw_intensity,verbose=False)
#Label the events
event_dict={'Control':1,'Tapping/Left':4,'Tapping/Right':3,'ExperimentEnds':2}
#Display the sequence of events 
plt.rcParams["figure.figsize"]=(10,6)
mne.viz.plot_events(events,event_id=event_dict,sfreq=raw_intensity.info['sfreq']);

As you may observe, each individual stimulus is separated to the others by long interstimulus interval. We can thus conclude that an event-related design was used for the studied measurement.

##### Question
Roughly, what is the average interstimulus interval of this design ?

Let's now vizualise the data. Run the next cell to display data obtained with the fNIRS system. 


In [ ]:
#Create a figure window
plt.rcParams["figure.figsize"]=(40,40)
#Plot a time-series of light intensity 
raw_intensity.plot(duration=100,n_channels=len(raw_intensity.ch_names))

The near-infrared light sent by a source (labelled "S" in our system) is absorbed by the HbO and HHb with a maximal absorption observed at the wavelengths 760nm and 850nm, respectively. The amounts of light absorbed by the two molecules are then detected by a detector  (labelled "D" in our system). As a consequence, each channel provides with two time-series of concentration changes, one for the HbO and the other for the HHb. 

Data are thus initially provided in a raw format of light intensity. This can be converted using the Beer-Lambert law (see equation below), into an optical density which in turn can be translated into concentrations change of HbO and HbR, as proxies of the brain activity.

\begin{equation}
    log_{10}(\frac{I_0}{I})=\epsilon l c
    \end{equation}

With $OD_\lambda$, $I_o$, I, $\epsilon$, c and l respectively corresponding to the absorbance, the incident light intensity, the transmitted light intensity, the moral absorption coefficient (in $M^-1 cm^-1$), the molar concentration (in M) and the optical pathlength for a given wavelength $\lambda$.

But since the human tissue is a strong scattering medium for the near-infrared light, the Beer–Lambert law cannot be directly applied to biological tissue. Subsequently, a modified version of the Beer-Lambert law that takes into account the light scattering has been proposed (see equation below).

\begin{equation}
OD_\lambda= log\frac{I_o}{I}=\epsilon_\lambda lcB + OD_{R\lambda}
\end{equation}

With B and $OD_{R\lambda}$ respectively corresponding to the pathlength correction factor and the oxygen independent light losses due to scattering in the tissue. For more information on the equation, see the reference: Delpy DT, Cope M, Zee P van der, Arridge S, Wray S, Wyatt J. Estimation of optical pathlength 
through tissue from direct time of flight measurements. Phys Med Biol 1988; 33: 1433-1442.

Run the next cell to convert raw intensity into optical density.

In [ ]:
#Convert raw intensity into optical density
raw_od=mne.preprocessing.nirs.optical_density(raw_intensity)

And then, run the cell below to convert optical density into concentration change.

In [ ]:
#Import required beer-lambert law related function
from mne.preprocessing.nirs import beer_lambert_law
#Convert optical density into concentration change
raw_haemo=beer_lambert_law(raw_od)

Now let's check the quality of the converted data. 

As highly perfused tissues, extra-cortical compartments such as the scalp, the skin, the meninges, or the cerebrospinal fluid can absorb the near-infrared light emitted by fNIRS sensors. In turn, the signal measured with fNIRS will confound changes of HbO and HbR concentrations that are evoked by a neural activation in the cortex or by intrinsic mechanism in extra-cortical compartments. For instance, a common change observed in fNIRS signal is the intrinsic variation of cardiac rhythm occurring in extra-cortical tissues.

Despite the presence of cardiac oscilliations in the signal is undesirable, it can be used wisely. In particular, the amount of physiological noise measured in a signal has been shown to reflect the optical coupling between the measuring channel and the scalp. Subsequently, a measure of the prominence of the cardiac oscilliations has been established to assess the scalp coupling index (SCI). For more information on the metric computation, see the references:
* Pollonini L et al., “PHOEBE: a method for real time mapping of optodes-scalp coupling in functional near-infrared spectroscopy” in Biomed. Opt. Express 7, 5104-5119 (2016). 
* Hernandez, Samuel Montero, and Luca Pollonini. "NIRSplot: a tool for quality assessment of fNIRS scans." Optics and the Brain. Optical Society of America, 2020.

Run the next cell to compute the SCI and plot the result.

In [ ]:
#Compute the SCI
%matplotlib inline
sci=mne.preprocessing.nirs.scalp_coupling_index(raw_od)
#Plot the SCI
fig,ax=plt.subplots()
ax.hist(sci)
ax.set(xlabel='Scalp Coupling Index',ylabel='Count',xlim=[0, 1]) 
plt.show()

##### Question
What is the number of bad channels (SCI < 6) ?

Run the next cell to mark the bad channels.

In [ ]:
import numpy as np
raw_od.info['bads'] = list(np.compress(sci < 0.6, raw_od.ch_names))

Let's now detect and correct the remaining component of the signal that originates from the cardiac oscillations. 

The cardiac oscilliations represent the major frequency component of the signal. 

Run the next cell to do decompose the power of the signal into its frequency components. Can you detect the frequency component corresponding to the cardiac oscillations ?  

In [ ]:
#Plot the hemoglobin concentration change
fig = raw_haemo.plot_psd(average=False)
fig.suptitle('Before filtering', weight='bold', size='x-large')
fig.subplots_adjust(top=0.88)

The power spectrum gives a frequency peak at 1,25 Hz which would correspond to approximately 1 beat / second. You may notice that this frequency corresponds to the cardiac rhythm. 

Fortunately, the noise induced by cardiac oscillations can be removed by filtering, since the corresponding frequency band is well distinguished from that of hemodynamic responses (< 0,5Hz). 

Based on the above information, apply a low-pass filter to only retain the hemodynamic responses in your signal. You may especially need to read the documentation on filters proposed by mne library (https://mne.tools/stable/generated/mne.filter.filter_data.html).

In [ ]:
raw_haemofiltered = #Your code here 

Run the next cell to plot your filtered signal.

In [ ]:
#Plot the hemoglobin concentration change
fig = raw_haemofiltered.plot_psd(average=False)
fig.suptitle('After filtering', weight='bold', size='x-large')
fig.subplots_adjust(top=0.88)

Let's now vizualise the data. Run the next cell to display the preprocessed time-series of one channel.

In [ ]:
#Plot the time-series of one channel
plt.rcParams["figure.figsize"]=(40,40)
raw_haemofiltered.pick(picks=[0,1]).plot(duration=100,n_channels=len(raw_intensity.ch_names),scalings=dict(hb0=1e-5,hbr=1e-5))

<h1><font color='black'>Part 2: diffusion data and tractography generation </font></h1> 

The preprocessing in DTI involves similar steps to what you saw in fMRI. We will thus tackle specific different steps to not repeat ourselves too much:
<img src="imgs/DTI_preprocessing.png" />
<p  style="text-align: center;"><i>Image from <a href="https://www.researchgate.net/publication/311246309_Imaging_analysis_of_Parkinson's_disease_patients_using_SPECT_and_tractography">Son, Seong-Jin, Mansu Kim, and Hyunjin Park. "Imaging analysis of Parkinson’s disease patients using SPECT and tractography." Scientific reports 6.1 (2016): 1-11.</a></i></p>

In other words, we will have you look at an example to generate tractogram. Here, a tractogram will be generated using a deterministic algorithm EuDX.  
  
Diffusion tensor imaging (DTI) is one of the most popular MRI techniques to describe the orientation of white matter fibers in brain research. The process of fiber tracking is called tractography. It allows for a virtual dissection and three-dimensional representation of white matter tracts. 
While we could still use FSL for the task, we will have you use <a href="https://dipy.org/">DIPY</a>, a Python package for computational neuroanatomy mainly focusing on diffusion MRI analysis.  
<br>
To generate a tractogram, we need to track the fibers, which is called fiber tracking.
<br>
Local fiber tracking is used to model white matter fibers by creating streamlines from local directional information. In order to perform local fiber tracking, you will apply the following three steps:
<p >
    <ol>
        <li style="font-size: 15px;">Extract directions from diffusion data</li> 
        <li style="font-size: 15px;">Identify when the tracking must stop</li>  
        <li style="font-size: 15px;">Select a set of locations from which to begin tracking</li>
    </ol>
</p>
Combining them will help you obtain a tractography reconstruction!

Ready? Let's go!

## 0. An additional package
We require two additional packages for this lab to work! They are only for visualization.
To install them, you will need to use the following commands:
```
conda install -c conda-forge fury
conda install -c conda-forge ipyvtklink 
``` 

### 1. Load the data 

In [ ]:
from dipy.core.gradients import gradient_table
from dipy.data import get_fnames
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti, load_nifti_data

hardi_fname, hardi_bval_fname, hardi_bvec_fname = get_fnames('stanford_hardi')
label_fname = get_fnames('stanford_labels')

data, affine, hardi_img = load_nifti(hardi_fname, return_img=True)
labels = load_nifti_data(label_fname)
bvals, bvecs = read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)
gtab = gradient_table(bvals, bvecs)

In [ ]:
print('data.shape: ',data.shape)
print('affine.shape: ',affine.shape)
print('hardi_img.shape: ',hardi_img.shape)

print('labels.shape: ',labels.shape)
print('bvals.shape: ',bvals.shape)
print('bvecs.shape: ',bvecs.shape)

In [ ]:
import os.path as op
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


print_dir_tree(op.split(hardi_fname)[0], max_depth=5)

### 2. Get the directions from the diffusion data set

#### 1. Defining the white matter region.
Before all else, you will need to visualize the labels above. Run the following cell to load the labels on FSLeyes:

In [ ]:
load(label_fname)
displayCtx.getOpts(overlayList[0]).cmap = 'brain_colours_spectrum'

Based on the values you read within, can you please fill in the cell below with the label corresponding to white matter?

In [ ]:
white_matter_value = ??? # Fill with the value you read in FSLeyes for white matter!

Great, let's now visualize the result of the mask, shall we? For this, let's generate the mask we obtained from above, using our best pal fslmaths ! 

In [ ]:
import os
output_name = 'extracted_wm'
cmd = 'fslmaths ' + label_fname + ' -thr ' + str(white_matter_value) + ' -uthr ' + str(white_matter_value) + ' -bin ' + output_name
os.system(cmd)
load(output_name)

That's nice, but we are missing something, aren't we? Look in the middle, there is a clear gap in purple. Why is that? If you have on top, you'll see it has a different label: 2. This is because this region is white matter, but it is also a sagittal slice of the **corpus callosum**. So we need to do something a bit different. Can you think of a way to modify the above fslmaths command to include both the white matter and the slice of corpus callosum ? :)

In [ ]:
lower_threshold = ??? # Select the lower bound to include both white matter and corpus callosum slice
upper_threshold = ?? # Select the upper bound to include both white matter and corpus callosum slice
output_name = 'extracted_wm_complete'
cmd = 'fslmaths ' + label_fname + ' -thr ' + str(lower_threshold) + ' -uthr ' + str(upper_threshold) + ' -bin ' + output_name
os.system(cmd)
load(output_name)

Beautiful! So you can see that these labels can be a bit tricky if you're not careful. Based on your above experience above, we will construct a mask in python directly. Please fill in the cell below the two values for:
- The white matter regions
- The corpus callosum slice

In [ ]:
corpus_callosum_slice_value = ??? # Fill with your value!
white_matter_value = ??? # Fill with your value !

total_white_matter = (labels == corpus_callosum_slice_value) | (labels == white_matter_value)

#### 2. Actually extracting fiber orientations: the orientation distribution function
Okay, now we have a mask to define our fibers. The next cell will be used to estimate the orientation distribution function at each voxel. Before going any further, let's ask why this is necessary. In your opinion, in a single *voxel* how many orientations can we have?
- [ ] Exactly one, since only one fiber is passing through the voxel
- [ ] One, as the orientation describes the voxel's orientation, not the fibers going through the voxel
- [ ] 26, since there are 26 neighbouring voxels with which a link is possible
- [ ] As many orientations as there are fibers going through the voxel

The issue can be summarized as resolving **intravoxel** fiber orientations of MR images.
To summarize these, we use an orientation distribution function, coined ODF.

<div class=\"warning\" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
    <span>
    <p style='margin-top:1em; text-align:center'><b>💡 Pay attention! 💡</b></p>
    <p style='text-indent: 10px;'>
        ODF really stands for orientation distribution function here, <b>not</b> ordinary differential function or anything else. Don't confuse the two!</p>
    </span>
</div>

We will not bore you with all mathematical details. What you need to know, however, is that this distribution function will rely on a special model, called the constant solid angle ODF model. The idea is the following: considering the distance from origin of the estimated distribution provides useful information. Here's what the solid angle looks like:
<img src="imgs/tractography/tileshop.jpeg"/>
<center>Left: pdf takes into account solid angle; Right: pdf does not take into account solid angle</center>
<i><center>Image taken from Aganj, Iman, et al. "Reconstruction of the orientation distribution function in single‐and multiple‐shell q‐ball imaging within constant solid angle." Magnetic resonance in medicine 64.2 (2010): 554-566.</center></i>


 

Let's now estimate the orientation distribution function of each voxel, using the CSA-ODF model. 

In [ ]:
from dipy.reconst.csdeconv import auto_response_ssst
from dipy.reconst.shm import CsaOdfModel
from dipy.data import default_sphere
from dipy.direction import peaks_from_model

# Single fiber response function: the measured signal of a single fiber
# sume: regions where there are single coherent fiber populations
# auto_response_ssst: calculate FA for a ROI of radii equal to roi_radii in the center of the volume
# and return the response function estimated in that region for the voxels with FA higher than 0.7
response, ratio = auto_response_ssst(gtab, data, roi_radii=10, fa_thr=0.7)

# Instantiate the Constant Solid Angle model
csa_model = CsaOdfModel(gtab, sh_order=6)

Now that we have our model, the orientation of tract segments can be extracted, looking at the peaks in the model.

In [ ]:
csa_peaks = peaks_from_model(csa_model, data, default_sphere,
                             relative_peak_threshold=.8,
                             min_separation_angle=45,
                             mask=total_white_matter, npeaks=5)

Notice in the above cell the following line:
```python
csa_peaks = peaks_from_model(..., npeaks=5)
```

This means that really, we extract per voxel five peaks at most. This is an important assumption. Depending on your voxel size, you might want to pay attention to this number!

To confirm this, let's have a look at the extracted peak values: 

In [ ]:
csa_peaks.peak_values.shape

Knowing the MR dimensions, you can see that we indeed have five peaks per voxel. Great! Let's visualize it now!

## Checkpoint: install an additional library

<div class="alert alert-warning">
    <p>Please go on the terminal and install the library <b>fury</b> by running:</p>
        
        conda install -c conda-forge fury
</div>

<div class="alert alert-info">
    <i>Make sure you are on the usual environment!</i>  
</div>

In [ ]:
from fury import actor, window, ui
from ipyvtklink.viewer import ViewInteractiveWidget

In [ ]:
scene = window.Scene()
slice_actor = actor.peak_slicer(csa_peaks.peak_dirs,
                            csa_peaks.peak_values,
                            affine=affine,mask=total_white_matter,
                            colors=None)
scene.add(slice_actor)

showm = window.ShowManager(scene, size=(900,900), reset_camera=False)
showm.initialize()
ViewInteractiveWidget(scene.GetRenderWindow())

So as you can see, the orientations do map out to the expected directions!

### 3. Set the stop criteria

Now, we need to setup our fiber tracking to stop it. What criterion should we use?
Well, we'll roughly use the idea that when we don't have enough evidence to know where a fiber could have gone, we stop tracking it.
In other words, if there are areas where the diffusion is totally unrestricted (goes in all directions), we have no clue as to where the fiber might continue. For this, we can threshold the tendency of our peaks to depend on a specific direction (anisotropy).<br>
More specifically, we will threshold the general fractional anisotropy of our data to decide when we should stop.

In [ ]:
from dipy.tracking.stopping_criterion import ThresholdStoppingCriterion

stopping_criterion = ThresholdStoppingCriterion(csa_peaks.gfa, .25)

Let's visualize a slice! 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

sli = csa_peaks.gfa.shape[2] // 2
plt.figure('GFA')
plt.subplot(1, 2, 1).set_axis_off()
plt.imshow(csa_peaks.gfa[:, :, sli].T, cmap='gray', origin='lower')

plt.subplot(1, 2, 2).set_axis_off()
plt.imshow((csa_peaks.gfa[:, :, sli] > 0.25).T, cmap='gray', origin='lower')

plt.savefig('gfa_tracking_mask.png')

### 4. Specify where to begin the fibers tracking

There are different ways to place seeds, ie starting points from which the fiber tracking is started. This depends on the pathways you might like to model! For example, if you only wanted to model the corpus callosum it would not be so interesting to place seeds in other regions of the brain. <br>
So that you understand what the output of the cell below will be, we must first explain what you'll extract in the cell below.<br>
The orientation of an image is described by its affine transformation, if you remember well. Let's call this affine $A$.
A seed point at the center of voxel $[i,j,k]$ will be represented as $[x,y,z, 1]= A \cdot [i,j,k,1]$<br>
In other words, you will get **coordinates in voxel space**. Note that there is one important assumption: the voxels here should be isotropic (the size of a voxel should be same along all directions).

In our specific case, we will start from a sagittal slice of the **corpus callosum**, the one with label 2 to be specific.
Please, create the mask (based on the labels above) to extract **only the slice of corpus callosum with label 2 as a mask**. You can refer to what we did above to do so. 

In [ ]:
from dipy.tracking import utils

seed_mask = ??? # Your code here to extract only the place of interest! 
seeds = utils.seeds_from_mask(seed_mask, affine, density=[2, 2, 2])

### 5. Bringing it all together and generating the streamlines
Let's see our ingredients:
- Seeds, generated above and starting from a slice of the corpus callosum
- A mask of regions where we should stop our fibers, based on anisotropy
- Peaks of ODF, at most five peaks per voxel

It remains now to combine all of these to bake so called streamlines (ie: fibers!). To do so, we will use the EuDX algorithm.

In [ ]:
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.streamline import Streamlines

# Initialization of LocalTracking. The computation happens in the next step.
streamlines_generator = LocalTracking(csa_peaks, stopping_criterion, seeds,
                                      affine=affine, step_size=.5)
# Generate streamlines object
streamlines = Streamlines(streamlines_generator)

In [ ]:
streamlines

Beautiful! Let's now visualize our streamlines!
Remember that they represent **only lines that start from the corpus callosum**! 

In [ ]:
from dipy.viz import colormap

# Prepare the display objects.
color = colormap.line_colors(streamlines)

streamlines_actor = actor.line(streamlines,
                               colormap.line_colors(streamlines))

# Create the 3D display.
scene = window.Scene()
scene.add(streamlines_actor)

showm = window.ShowManager(scene, size=(900,900), reset_camera=False)
showm.initialize()
ViewInteractiveWidget(scene.GetRenderWindow())

# Save still images for this static example. Or for interactivity use
#window.record(scene, out_path='tractogram_EuDX.png', size=(800, 800))
#if interactive:
#    window.show(scene)

### 6. The impact of seeds

If you remember, we did state that our seeds be restricted only to the slice of interest. Now, what would happen if we went for the full white matter? Obviously it would be more expensive, but let's do it for the sake of curiosity nonetheless! (Note that the display function might struggle a bit, that's fine)### 6. Store the streamlines

In [ ]:
seeds_t = utils.seeds_from_mask(total_white_matter, affine, density=[2, 2, 2])

In [ ]:
# Initialization of LocalTracking. The computation happens in the next step.
streamlines_generator_t = LocalTracking(csa_peaks, stopping_criterion, seeds_t,
                                      affine=affine, step_size=.5)
# Generate streamlines object
streamlines_t = Streamlines(streamlines_generator_t)

In [ ]:
# Prepare the display objects.
color = colormap.line_colors(streamlines_t)

streamlines_actor = actor.line(streamlines_t,
                               colormap.line_colors(streamlines_t))

# Create the 3D display.
scene = window.Scene()
scene.add(streamlines_actor)

showm = window.ShowManager(scene, size=(900,900), reset_camera=False)
showm.initialize()
ViewInteractiveWidget(scene.GetRenderWindow())

### 7. Store the streamlines into a trackvis file

What if we wanted to save the result as a file? Well, you can! For this, we need to save it to a special format, the TrackVis (.trk) format.

Remember: our goal was to generate the streamlines. It is these streamlines that we therefore want to save! :) 
Let's do it! 

In [ ]:
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import save_tractogram, save_trk

# This is for the cc slice tractogram
sft = StatefulTractogram(streamlines, hardi_img, Space.RASMM)
save_trk(sft, "tractogram_EuDX.trk", streamlines)

# This is for the whole wm tractogram
sft_t = StatefulTractogram(streamlines_t, hardi_img, Space.RASMM)
save_trk(sft_t, "tractogram_full_wm_EuDX.trk", streamlines_t)

If you want to visualize it all, you can install <a href="http://trackvis.org/download/">TrackVis</a> and open the file from within! (TrackVis is free, no worries). It will be interactive and in 3D! Pretty cool huh?<br> 

### Conclusions

Let's sum up what we've seen. For a successfull tractography generation, we need the following: 
<table>
    <tr>
        <th>Ingredient</th>
        <th>Role</th>
        <th>How is it created?</th>
    </tr>
    <tr>
        <td>Seeds</td>
        <td>Define starting point of tract propagation.</td>
        <td>Can be done randomly or according to some mask of interest</td>
    </tr>
    <tr>
        <td>Diffusion directions</td>
        <td>Define the local diffusion in a voxel, for all voxels of interest</td>
        <td>Can be done with CSA-ODF or other methods such as e.g structure tensor</td>
    </tr>
    <tr>
        <td>Stopping criteria</td>
        <td>Defines where the tract continues or stops.</td>
        <td>Can be done based on anatomy, information of diffusion direction, combination of both...</td>
    </tr>
    <tr>
        <td>A tracking algorithm</td>
        <td>Combines all ingredients above to generate streamlines</td>
        <td>Line propagation techniques to grow from seed region, or probabilistic with a pdf of fiber orientations.</td>
    </tr>
</table> 

<img src="imgs/tractography/4qamin.jpg"/>

Each of the ingredients can be changed for a different flavour. You can explore <a href="https://dipy.org/tutorials/">DIPY's tutorials</a> to get an idea of the changes you can operate. Feel free to play around!

# Part 3: Connectivity analysis based on tractography 

Now that you've seen how to generate streamlines, we will look at their analysis. 
For example, how do we find the streamlines those pass through or do not pass through some region of the brain? How do we count streamlines based on a start and end point in the brain? How do we count the number of streamlines that pass through each voxel of some image?



### 1. Generating the streamlines to work with

Obviously, the first part to analyse streamlines is...to have streamlines!
For this, we refer you to the previous part on tractography. What we will do here is create a tractography with seeds spanning the entire white matter. As you've already saved the streamlines as a file, we will load it directly! To do so, we need a reference anatomy (have a look <a href="https://dipy.org/documentation/1.5.0/examples_built/streamline_formats/">here</a> for a more complete treatment of DIPY's loading philosophy)

In [ ]:
from dipy.io.streamline import load_tractogram
from dipy.core.gradients import gradient_table
from dipy.data import get_fnames
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti, load_nifti_data

import nibabel as nib

hardi_fname, _, _ = get_fnames('stanford_hardi')
reference_anatomy = nib.load(hardi_fname)
t1_fname = get_fnames('stanford_t1')
t1_data = load_nifti_data(t1_fname)

In [ ]:
tractogram = load_tractogram("tractogram_full_wm_EuDX.trk", reference_anatomy)

# Important: some stream lines might be invalid. Let's rid ourselves of them :)
tractogram.remove_invalid_streamlines()
streamlines = tractogram.streamlines
affine = tractogram.affine

### 2.1 Count the number of streamlines passing through a given ROI

How many streamlines pass through the slice of corpus callosum you saw in the previous section? To figure this out, we simply need to extract back the corpus callosum mask from the label file. Let's do it!

In [ ]:
# Let's get back the mask
label_fname = get_fnames('stanford_labels')
labels = load_nifti_data(label_fname)

# generate the mask for the sagittal slice of corpus callosum
cc_slice = ??? # Put here your code to do it!

Now, we will use a function from utils, the target function.
It takes a set of streamlines, a region of interest (ROI) and a boolean flag.
If the flag is set to true, the function returns *only* the streamlines passing through the mask.
If the flag is set to false, it returns *only* streamlines not passing through the mask.

Obviously, these two results should sum up to exactly our number of streamlines, otherwise we have an issue! 

In [ ]:
from dipy.tracking import utils
from dipy.tracking.streamline import Streamlines

# These are our streamlines of interest: those going through our mask!
cc_streamlines = utils.target(streamlines, affine, cc_slice)
cc_streamlines = Streamlines(cc_streamlines)

# These are all other streamlines
other_streamlines = utils.target(streamlines, affine, cc_slice,
                                 include=False)
other_streamlines = Streamlines(other_streamlines)

# Just a quick check: the two sets of streamlines should sum up to exactly the number of streamlines!
assert len(other_streamlines) + len(cc_streamlines) == len(streamlines)

So, how many streamlines pass through the slice of corpus callosum?
- [ ] Half of all streamlines
- [ ] All of them
- [ ] None of them
- [ ] About ten percent of them

For the sake of completeness, how about visualizing the fibers that pass through the slice of corpus callosum? 

In [ ]:
from fury import actor, window, ui, colormap as cmap
from ipyvtklink.viewer import ViewInteractiveWidget

color = cmap.line_colors(cc_streamlines)
cc_streamlines_actor = actor.line(cc_streamlines,
                                  cmap.line_colors(cc_streamlines))
cc_ROI_actor = actor.contour_from_roi(cc_slice, color=(1., 1., 0.),
                                      opacity=0.5, affine=affine)

vol_actor = actor.slicer(t1_data, affine)

vol_actor.display(x=40)
vol_actor2 = vol_actor.copy()
vol_actor2.display(z=35)

# Add display objects to canvas
scene = window.Scene()
scene.add(vol_actor)
scene.add(vol_actor2)
scene.add(cc_streamlines_actor)
scene.add(cc_ROI_actor)

showm = window.ShowManager(scene, size=(900,900), reset_camera=False)
showm.initialize()
ViewInteractiveWidget(scene.GetRenderWindow())

### 2.2 Finding out which regions of the brain are connected by these streamlines

Now we'll use the streamlines to build a connectivity matrix. Why? The idea is simple: we want to know where streamlines start and where they end in the brain. Since the brain is labelled, we can figure this out by alternatively asking how many fibers go from a region to another. Answer the following to build up some intuition:

- [ ] We are modelling the brain as a graph. The nodes are the fibers, and the edges are the regions.
- [ ] We are modelling the brain as a graph. The nodes are the regions, and the edges are the number of fibers between pair of nodes.

Now, all is good and well. But let's go a bit deeper. If we count the number of fibers going from region A to B, for all pairs of region, and we set up our seeds to grow from region R to all other regions, what do you expect to happen?
- [ ] Region R's line in the connectivity matrix will describe all tracts of the tractogram, because they exclusively grow from it to other regions?
- [ ] All regions except region R will have zero tract going to other regions, because they were not used as seed to grow tractograms from?
- [ ] We cannot accurately decide without extensive knowledge of the tracking algorithm, the seed mask itself is an insufficient information to know if there is any directionality involved in the reconstruction

In [ ]:
# connectivity_matrix: takes a set of streamlines and an array of labels as arguments
# returns the number of streamlines that start and end at each pair of labels and return streamlines grouped by their endpoints
M, grouping = utils.connectivity_matrix([sl[0::len(sl)-1] for sl in cc_streamlines if len(sl)-1 > 0], affine,
                                        labels.astype(np.uint8),
                                        return_mapping=True,
                                        mapping_as_streamlines=True)
# we are interested in connections between GM regions, label 0 represents background and 1,2 represent WM
# DISCARD THE FIRST THREE ROW AND COLUMNS OF THE CONNECTIVITY MATRIX
M[:3, :] = 0
M[:, :3] = 0

In [ ]:
# display the matrix by matplotlib
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

plt.imshow(np.log1p(M), interpolation='nearest')
plt.savefig("connectivity.png")

Let's find the maximum (besides diagonal elements), and where they are located: 

In [ ]:
np.where(M == np.max(M-np.eye(M.shape[0]) * np.diag(M)))

Remember we are dealing here with **labels**. We can extract the label infos, straight from the label file:

In [ ]:
import pandas as pd
from os.path import dirname, abspath
import os.path as op

region_labels = pd.read_csv(op.join(dirname(abspath(label_fname)), 'label_info.txt'))
region_labels

Let's find the region with label 11

In [ ]:
region_labels[region_labels["new label"] == ???] # set the proper value here!

And the region with label 54 

In [ ]:
region_labels[region_labels["new label"] == ???] # set the proper value here!

So, more tracts connect the left superior frontal gyrus and the right superior frontal gyrus than other pair of regions. In this particular case, the matrix was symetric. Again remember: the exact interpretation is **hard** and a very active area of research, so you'll need to really be sure of what your tracking algorithm does before making any claim :)

### 2.3 Representing the spatial distribution of a track by counting the density of streamlines in each voxel

In [ ]:
lr_superiorfrontal_track = grouping[11, 54]
shape = labels.shape
dm = utils.density_map(lr_superiorfrontal_track, affine, shape)

In [ ]:
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import save_trk
from dipy.io.image import load_nifti_data, load_nifti, save_nifti

# Save density map
save_nifti("lr-superiorfrontal-dm.nii.gz", dm.astype("int16"), affine)

lr_sf_trk = Streamlines(lr_superiorfrontal_track)

# Save streamlines
_, _, hardi_img = load_nifti(hardi_fname, return_img=True)
sft = StatefulTractogram(lr_sf_trk, hardi_img, Space.VOX)
save_trk(sft, "lr-superiorfrontal.trk", bbox_valid_check=False)

This ends this short tractography tutorial! Again, do not hesitate to explore more on DIPY's website if you're interested :)
We strongly encourage you to use TrackVis for visualization of tractograms as it's really made for it and is much more intuitive to use for this purpose than FSLeyes.

<div class="alert alert-success">
<p><b>Congratulations for having finished this part of the laboratory! </b></p>
</div>